### Environment.state and Constraints

Last updated: 01/22/2020

In [5]:
import numpy as np
import pandas as pd

from marmot import Environment, Agent, process

#### State Data

In [2]:
# The marmot.Environment class can also be configured with time series that represent the
# state of the environment throughout time. This dataset can then be used to dynamically
# constrain tasks as they occur throughout the simulation.


# Load data
data = pd.read_csv("test_data.csv")\
         .set_index("datetime")\
         .to_records()  # The input data type should be np.ndarray

# To create an Environment with state data:
env = Environment("Test Environment", state=data)

# The current state forecast can be accessed through env.state:
env.state

rec.array([( 1, 12, 1,  True), ( 2,  6, 1,  True), ( 3,  7, 1, False),
           ( 4,  4, 1, False), ( 5,  3, 1, False), ( 6, 12, 2, False),
           ( 7, 12, 2, False), ( 8, 12, 2, False), ( 9,  5, 2, False),
           (10,  6, 2, False), (11,  7, 2, False), (12,  3, 2, False),
           (13,  2, 3, False), (14,  3, 3, False), (15,  4, 3, False),
           (16,  5, 3, False), (17,  1, 2, False), (18,  2, 2, False),
           (19,  5, 2, False), (20,  8, 2, False), (21,  9, 2, False),
           (22,  8, 2, False), (23,  5, 3, False), (24,  4, 3, False),
           (25,  3, 3, False), (26,  2, 3,  True), (27,  1, 3,  True),
           (28,  8, 3,  True), (29,  7, 2,  True), (30,  6, 2,  True),
           (31,  5, 2,  True), (32,  9, 2,  True), (33,  2, 2,  True),
           (34, 15, 2,  True), (35,  4, 2,  True), (36,  8, 1,  True),
           (37,  8, 1,  True), (38,  8, 1,  True), (39,  9, 1, False),
           (40,  5, 1, False), (41,  5, 1, False), (42,  4, 1, False),
      

In [3]:
# As the environment progresses througout time, the upcoming forecast adjusts:
env.timeout(10)
env.run()
env.state

rec.array([(11,  7, 2, False), (12,  3, 2, False), (13,  2, 3, False),
           (14,  3, 3, False), (15,  4, 3, False), (16,  5, 3, False),
           (17,  1, 2, False), (18,  2, 2, False), (19,  5, 2, False),
           (20,  8, 2, False), (21,  9, 2, False), (22,  8, 2, False),
           (23,  5, 3, False), (24,  4, 3, False), (25,  3, 3, False),
           (26,  2, 3,  True), (27,  1, 3,  True), (28,  8, 3,  True),
           (29,  7, 2,  True), (30,  6, 2,  True), (31,  5, 2,  True),
           (32,  9, 2,  True), (33,  2, 2,  True), (34, 15, 2,  True),
           (35,  4, 2,  True), (36,  8, 1,  True), (37,  8, 1,  True),
           (38,  8, 1,  True), (39,  9, 1, False), (40,  5, 1, False),
           (41,  5, 1, False), (42,  4, 1, False), (43,  3, 1, False),
           (44,  2, 1, False), (45,  8, 1, False), (46,  9, 2, False),
           (47,  7, 1, False), (48, 12, 1, False), (49,  4, 1, False),
           (50,  8, 1, False)],
          dtype=[('datetime', '<i8'), ('winds

In [4]:
# Reset env, create an agent and register it
env = Environment("Test Environment", state=data)
agent = Agent("Test Agent")
env.register(agent)

# A task that is submitted without a constraint will be processed immediately
agent.task("Run", duration=10)
env.run()
env.now

10

#### Constraints

In [8]:
# There are 6 Constraints currently included in marmot that can be used within an agent.task()
# These return a callable instance that can be used to compare arrays in a repeatable way:
# - gt(val): greater than val
# - ge(val): greater than or equal to val
# - lt(val): less than val
# - le(val): less than or equal to val
# - true(): evaluates the input array
# - false(): evaluates the opposite of the input array

from marmot import gt, ge, lt, le, true, false

constraint = lt(10)            # Create the constraint, "Less than 10"
constraint(data["windspeed"])  # Compare against the windspeed column of data
                               # Returns a boolean array where that constraint evaluates to True

array([False,  True,  True,  True,  True, False, False, False,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True, False,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True, False,  True,  True])

In [11]:
# These constraints can be used within the agent.task method to apply a constraint to the task:


# Reset env, create an agent and register it
env = Environment("Test Environment", state=data)
agent = Agent("Test Agent")
env.register(agent)

# Create task:
agent.task("Run", duration=10, constraints={"windspeed": lt(10)})
env.run()
env.now    # The applied constraint delayed the completion of the task until time = 18

18

In [12]:
env.logs   # There was a delay of 8 units at the beginning of the task.
           # By default, a constrained task must be completed within a state window
           # where that state always remains True. Looking above at the boolean array that was returned for windspeed < 10,
           # the first window of 10 successive True values occurs at the 9th index. The task is delayed
           # until this point and then will proceed.

[{'agent': 'Test Agent',
  'action': 'Delay',
  'duration': 8.0,
  'level': 'ACTION',
  'time': 8},
 {'agent': 'Test Agent',
  'action': 'Run',
  'duration': 10.0,
  'level': 'ACTION',
  'time': 18}]

In [13]:
# Repeating the above task, but with the "suspendable" flag


# Reset env, create an agent and register it
env = Environment("Test Environment", state=data)
agent = Agent("Test Agent")
env.register(agent)

# Create task:
agent.task("Run", duration=10, constraints={"windspeed": lt(10)}, suspendable=True)
env.run()
env.now    # The applied constraint delayed the completion of the task until time = 14

14

In [14]:
env.logs   # The suspendable flag allows the task to be interrupted when constraints are
           # violated and completed at a later point. Looking above at the boolean array that was returned for windspeed < 10,
           # the agent is briefly delayed at the first index, then is able to perform the task for 4 time steps.
           # After another delay of 3 time steps, the task is completed at time = 14

[{'agent': 'Test Agent',
  'action': 'Delay',
  'duration': 1.0,
  'level': 'ACTION',
  'time': 1},
 {'agent': 'Test Agent',
  'action': 'Run',
  'duration': 4.0,
  'level': 'ACTION',
  'time': 5},
 {'agent': 'Test Agent',
  'action': 'Delay',
  'duration': 3.0,
  'level': 'ACTION',
  'time': 8},
 {'agent': 'Test Agent',
  'action': 'Run',
  'duration': 6.0,
  'level': 'ACTION',
  'time': 14}]

In [20]:
# There can be multiple constraints applied to each task:
# Any constraints that the key doesn't match a column in env.state are ignored.


# Reset env, create an agent and register it
env = Environment("Test Environment", state=data)
agent = Agent("Test Agent")
env.register(agent)

# Create task:

constraints = {
    "windspeed": lt(10),
    "waveheight": lt(3),
    "whales": false(),
    "water_temp": gt(50)  # Will be ignored as env.state doesn't contain data for "water_temp"
}

agent.task("Run", duration=10, constraints=constraints, suspendable=True)
env.run()
env.now    # The applied constraints delayed the completion of the task until time = 19

19

In [21]:
env.logs

[{'agent': 'Test Agent',
  'action': 'Delay',
  'duration': 2.0,
  'level': 'ACTION',
  'time': 2},
 {'agent': 'Test Agent',
  'action': 'Run',
  'duration': 3.0,
  'level': 'ACTION',
  'time': 5},
 {'agent': 'Test Agent',
  'action': 'Delay',
  'duration': 3.0,
  'level': 'ACTION',
  'time': 8},
 {'agent': 'Test Agent',
  'action': 'Run',
  'duration': 4.0,
  'level': 'ACTION',
  'time': 12},
 {'agent': 'Test Agent',
  'action': 'Delay',
  'duration': 4.0,
  'level': 'ACTION',
  'time': 16},
 {'agent': 'Test Agent',
  'action': 'Run',
  'duration': 3.0,
  'level': 'ACTION',
  'time': 19}]